# **Batch Data Processing using Spark on Databricks**

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import urllib

> ### Extracting the batch data

*Reading the csv file containing the AWS keys to the databricks*

In [ ]:
file_type = "csv"
first_row_is_header = "true"
delimiter = ","
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

*Mounting S3 bucket to the databricks*

In [ ]:
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

AWS_S3_BUCKET = "user-12f7a43505b1-bucket"
MOUNT_NAME = "/mnt/pinterest_s3_mount"
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

>### Creating Dataframes

*- Creating the following three dataframes: `df_pin`, `df_geo` and `df_user`*

In [ ]:
topics =  {'df_pin':'12f7a43505b1.pin', 'df_geo':'12f7a43505b1.geo', 'df_user':'12f7a43505b1.user'}
for df_name, topic in topics.items():
    file_location = f"/mnt/pinterest_s3_mount/topics/{topic}/partition=0/*.json" 
    file_type = "json"
    infer_schema = "true"
    df_name = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(file_location)

>### Data Cleaning

#### *Data cleaning for df_pin*
 - Removing duplicate rows in the dataframe
 - Renaming the column `index` to `ind`
 - Re-ordering the column names in the dataframe
 - Replacing the values of `follower_count` column wherever necessary and hence converting the column into a integer data type
 - Removing any additional strings from the `save_location` column
 - Replacing all the NA with `None`
- Dropping the rows where all columns have null values


In [ ]:
df_pin = df_pin.dropDuplicates()
df_pin = df_pin.withColumnRenamed('index', 'ind')
df_pin = df_pin.select("ind", "unique_id", "title", "description", "follower_count", "poster_name", 
"tag_list", "is_image_or_video", "image_src", "save_location", "category", "downloaded")
df_pin = df_pin.withColumn('follower_count', regexp_replace('follower_count', '[%k]', '000'))
df_pin = df_pin.withColumn('follower_count', regexp_replace('follower_count', '[%M]', '000000'))
df_pin = df_pin.withColumn('follower_count', regexp_replace('follower_count', '[%User Info Error%]', ''))
df_pin = df_pin.withColumn('follower_count', df_pin['follower_count'].cast(IntegerType()))
df_pin = df_pin.withColumn('save_location', regexp_replace('save_location', 'Local save in *', ''))
df_pin.na.fill('None', ['is_image_or_video', 'image_src'])
df_pin.na.drop(how = "all")

#### *Data cleaning for df_geo*
 - Removing duplicate rows in the dataframe
 - Creating new column `coordinates` with the values to be the array of `latitude` and `longitude` column and deleting these two columns
 - Converting the `timestamp` column into a timestamp data type
 - Re-ordering the column names in the dataframe

In [ ]:
#df_geo
df_geo = df_geo.dropDuplicates()
df_geo = df_geo.withColumn("coordinates", array(col("latitude"), col("longitude")))
df_geo = df_geo.drop('latitude', 'longitude')
df_geo = df_geo.withColumn("timestamp", df_geo["timestamp"].cast(TimestampType()))
df_geo = df_geo.select("ind", "country", "coordinates", "timestamp")

#### *Data cleaning for df_user*
 - Removing duplicate rows in the dataframe
 - Creating new column `user_name` by combining the `first_name` and `last_name` column and deleting these two columns 
 - Converting the `date_joined` column into a timestamp data type
 - Re-ordering the column names in the dataframe

In [ ]:
#df_user
df_user = df_user.dropDuplicates()
df_user = df_user.withColumn("user_name", concat(col("first_name"), lit(" "), col("last_name")))
df_user = df_user.drop("first_name", "last_name")
df_user = df_user.withColumn('date_joined', df_user['date_joined'].cast(TimestampType()))
df_user = df_user.select("ind", "user_name", "age", "date_joined")

>### Data Analysis

*The most popular Pinterest category, people post to, based on their country.*

In [ ]:
df_pin.join(df_geo, 'ind').groupBy('country').agg(mode('category').alias('category'), count_distinct('category').alias('category_count')) \
  .orderBy('country').show()

*Number of posts each category had between 2018 and 2022 and most popular category in each year.*

In [ ]:
df_pin.join(df_geo, 'ind').groupBy(year('timestamp').alias('post_year')).agg(mode('category').alias('category'), count_distinct('category') \
  .alias('category_count')).select('post_year', 'category', 'category_count').where(col('post_year').between('2018', '2022')) \
  .orderBy('post_year').show()

*Users with most followers in each country*

In [ ]:
#STEP 1
df_pin.join(df_geo, 'ind').groupBy('country').agg(max(struct('follower_count', 'poster_name')).alias('max')) \
  .selectExpr('country', 'max.poster_name', 'max.follower_count').orderBy(col('follower_count').desc()).show()
#STEP 2
df_pin.join(df_geo, 'ind').groupBy('country').agg(max(struct('follower_count', 'poster_name')).alias('max')) \
  .selectExpr('country', 'max.follower_count').orderBy(col('follower_count').desc()).show()

*The most popular category people post to, based on the age groups - 18-24, 25-35, 36-50, +50*

In [ ]:
df_pin.join(df_user, 'ind').withColumn('age_group', expr("""case
 when age BETWEEN 18 AND 24 THEN '18-24'
 when age BETWEEN 25 AND 35 THEN '25-35'
 when age BETWEEN 36 AND 50 THEN '36-50'
 when age> 50 THEN '+50'
 END
 """)).groupBy('age_group').agg(mode('category').alias('category'), count_distinct('category').alias('category_count')).show()

*The median follower count for users in the age groups, 18-24, 25-35, 36-50, +50*

In [ ]:
df_pin.join(df_user, 'ind').withColumn('age_group', expr("""case
 when age BETWEEN 18 AND 24 THEN '18-24'
 when age BETWEEN 25 AND 35 THEN '25-35'
 when age BETWEEN 36 AND 50 THEN '36-50'
 when age> 50 THEN '+50'
 END
 """)).groupBy('age_group').agg(median('follower_count').alias('median_follower_count')).show()

*How many users have joined between 2015 and 2020.*

In [ ]:
df_user.groupBy(year('date_joined').alias('post_year')).agg(count_distinct('ind').alias('number_users_joined')) \
    .select('post_year', 'number_users_joined').where(col('post_year').between('2015', '2020')).show()

*The median follower count of users who have joined between 2015 and 2020.*

In [ ]:
df_user.join(df_pin, 'ind').groupBy(year('date_joined').alias('post_year')).agg(median('follower_count').alias('median_follower_count')) \
.select('post_year', 'median_follower_count').where(col('post_year').between('2015', '2020')).show()

*The median follower count of users who have joined between 2015 and 2020, based on age group that they are part of.*

In [ ]:
df_pin.join(df_user, 'ind').withColumn('age_group', expr("""case
 when age BETWEEN 18 AND 24 THEN '18-24'
 when age BETWEEN 25 AND 35 THEN '25-35'
 when age BETWEEN 36 AND 50 THEN '36-50'
 when age> 50 THEN '+50'
 END
 """)).groupBy(year('date_joined').alias('post_year'), 'age_group').agg(median('follower_count').alias('median_follower_count')) \
.select('age_group','post_year', 'median_follower_count').where(col('post_year').between('2015', '2020')).orderBy('age_group', 'post_year').show()